In [2]:
from obj_preprocessing import GCS_preprocessing
from google.cloud import aiplatform as aip
from tabulate import tabulate

In [3]:
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
REGION= "europe-west3"
REPO_NAME = "repo-demo3"
SERVICE_ACCOUNT = "944308723981-compute@developer.gserviceaccount.com"
BUCKET = "ccai-storage"
PIPELINE_NAME = "automl_pipeline"
YAML_NAME = f"{PIPELINE_NAME}.yml"
PIPELINE_ROOT = f"gs://{BUCKET}/pipeline_root/"
DISPLAY_NAME = PIPELINE_NAME.replace("_", "-")
NOTEBOOK = "automl"
DATANAME = "datasetnlp"
FILE_PATH = 'test_file.parquet'
FOLDER = 'make_prediction'
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
TABLE_ID = "testdatabq"
TEXT_COLUMN = 'body_pre'
LOCATION = "europe-west3"
NUM_DOC = 20
RANDOM_SEED=123
#BQ_SOURCE = "bq://gcp-ccai-auto-ml-contactcenter.datasetnlp.stepfinalbq"

OUTPUT_FINAL = 'step_final_bq.parquet'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'


In [5]:
processor = GCS_preprocessing(bucket=BUCKET,
                              folder=FOLDER,
                              file_path=FILE_PATH,
                              parquet_file_name=OUTPUT_FINAL,
                              num_doc=NUM_DOC, 
                              random_seed=RANDOM_SEED,
                              project_id=PROJECT_ID,
                              dataset_id=DATANAME,
                              table_id=TABLE_ID,
                              location=LOCATION,
                              text_column=TEXT_COLUMN,
                              pipeline_root=PIPELINE_ROOT,
                              overwrite=True)


instances = processor.process()


[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


INPUT DATAFRAME
                  uri                                                url  \
0  2023-12-196637772  https://www.repubblica.it/il-gusto/2023/12/19/...   
1  2023-12-196631748  https://www.libero.it/magazine/news/grande-fra...   
2         7893333817  https://www.tgcom24.mediaset.it/lifestyle/beau...   
3         7893333963  https://www.corriere.it/lodicoalcorriere/index...   
4  2023-12-196630770  https://www.repubblica.it/tecnologia/2023/12/1...   

                                               title  \
0  Tutto il gusto dell'Abruzzo d'inverno, tra cas...   
1  GF, Varrese sotto attacco: pesanti critiche da...   
2  Natale, il make up più chic delle Feste è ispi...   
3  La rivalità Goggia-Brignone    grande romanzo ...   
4                        Tech: i personaggi del 2023   

                                                body        date      time  \
0  Tra ciaspolate, rifugi, alcune tra le piste mi...  2023-12-18  23:26:23   
1  Scrittrice, copywriter, editor e pub

In [6]:
aip.Endpoint.list(filter=f'labels.notebook={NOTEBOOK}')
endpoint = aip.Endpoint.list(filter=f'labels.notebook={NOTEBOOK}')[0]
print(f"ENDPOINTS: \n{endpoint.display_name}\n")
prediction = endpoint.predict(instances = instances) # or instances = newobs

dictionary=prediction.predictions[2]
# Convert the dictionary into a list of tuples
table_data = list(zip(dictionary['classes'], dictionary['scores']))

# Print the table
print("PREDICTIONS")
print(tabulate(table_data, headers=['Class', 'Score'], tablefmt='grid'))
#print(f"PREDICTIONS: \n {prediction.predictions[0]}")


ENDPOINTS: 
automl_datasetnlp_20240325224945

PREDICTIONS
+------------------------+-------------+
| Class                  |       Score |
+========================+=============+
| Technology             | 0.00222796  |
+------------------------+-------------+
| Sports                 | 0.0321837   |
+------------------------+-------------+
| Science                | 0.000346805 |
+------------------------+-------------+
| Politics               | 0.86633     |
+------------------------+-------------+
| Health                 | 0.000567793 |
+------------------------+-------------+
| Environment            | 0.000287425 |
+------------------------+-------------+
| Business               | 0.040143    |
+------------------------+-------------+
| Arts and Entertainment | 0.0579132   |
+------------------------+-------------+
